In [80]:
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import pandas as pd
from selenium import webdriver

In [81]:
bbc_url = 'https://www.bbc.com'
bbc_url_news = 'https://www.bbc.com/news'
world_url = 'https://edition.cnn.com/world'
politics_url = 'https://edition.cnn.com/politics'
health_url = 'https://edition.cnn.com/health'
entertainment_url = 'https://edition.cnn.com/entertainment'

In [82]:
def getParseUrl(link, genre):
    driver = getDriver()
    driver.get(link + '/news')
    button = driver.find_element_by_link_text(genre)
    button.click()
    webContent = driver.page_source
    data = BeautifulSoup(webContent, 'html.parser')
    driver.close()
    return data

In [83]:
def getParseUrlStatic(link):
    response = urllib.request.urlopen(link)
    webContent = response.read()
    
    data = BeautifulSoup(webContent, 'html.parser')
    return data

In [84]:
def getNewsLinks(data):
    headlines = data.find_all(class_ = "gs-c-promo-heading")
    links = []
    for headline in headlines:
        try:
            link = headline["href"]
            if link[:5] != 'https':
                link = bbc_url + link
            links.append(link)
        except:
            print("sed")
    return links

In [86]:
def getDriver():
    driver = webdriver.Chrome(executable_path=r'C:\Users\DELL\Downloads\chromedriver_win32\chromedriver')
    return driver

In [87]:
def getStringFromTags(tag):
    data = tag.contents
    string = ""
    for content in data:
        try:
            string += content
        except:
            string += getStringFromTags(content)
    return string

In [101]:
def getNewsFromLink(link, genre):
    news = {}
    try:
        data = getParseUrlStatic(link)
        news = {}
        news['heading'] = data.find('h1', {'id': 'main-heading'}).contents[0]
        news['content'] = ""
        divs = data.find_all(True, {'class': ["ssrcss-uf6wea-RichTextComponentWrapper e1xue1i87", "Paragraph__component BasicArticle__paragraph BasicArticle__pad", "Paragraph__component"]})
        for div in divs:
            news['content'] += getStringFromTags(div)
        news['link'] = link
        news['genre'] = genre
    except:
        print("sed")
    return news

In [103]:
def getDataFromUrl(url, genre):
    data = getParseUrl(url, genre)
    links = getNewsLinks(data)
    news = [getNewsFromLink(link, genre) for link in links]
    return news

In [113]:
news = []
for genre in ['Climate', 'World', 'Asia', 'Tech', 'Business', 'Science', 'Entertainment & Arts']:
    cur_news = getDataFromUrl(bbc_url, genre)
    for new in cur_news:
        news.append(new)

sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed
sed


In [114]:
len(news)

290

In [115]:
pd.DataFrame(news).dropna()

,heading,content,link,genre
0,Climate change: Should green campaigners put m...,China will be urged at the UN next week to spe...,https://www.bbc.com/news/science-environment-5...,Climate
1,Climate change: Should green campaigners put m...,China will be urged at the UN next week to spe...,https://www.bbc.com/news/science-environment-5...,Climate
2,Climate change: UN warning over nations' clima...,"Despite all the promises to take action, the w...",https://www.bbc.com/news/science-environment-5...,Climate
3,Carbon emitters 'failing to disclose climate r...,A lack of detail in financial reporting will d...,https://www.bbc.com/news/business-58589914,Climate
4,Climate change: World now sees twice as many d...,The number of extremely hot days every year wh...,https://www.bbc.com/news/science-environment-5...,Climate
...,...,...,...,...
278,Manchester music cafe brings joy to people wit...,,https://www.bbc.com/news/uk-england-manchester...,Entertainment & Arts
279,Naomi Campbell on being a mother: I have a dre...,,https://www.bbc.com/news/entertainment-arts-58...,Entertainment & Arts
280,The Wanted's Tom Parker opens up about his can...,,https://www.bbc.com/news/entertainment-arts-58...,Entertainment & Arts
281,Andrew Neil on GB News: 'I was minority of one',,https://www.bbc.com/news/uk-58592178,Entertainment & Arts
